In [1]:
import os
import json
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torch.optim import Adam
from tqdm import tqdm

# data_path = "~/scratch/CS_6476_project_code/data/processed_data/"
data_path = "../data/processed_data/"

%load_ext autoreload
%autoreload 2

from utils import create_image_labels_list
from utils import split_data

In [17]:
seed = 42

# create image labels from annotations
image_labels, _ = create_image_labels_list(data_path)
print(image_labels[0:3])

[('0001c184bea6429b8372f7ea88e4fe1c.jpg', 0), ('0001d9cf1d0f488f83efca4cbc87edbf.jpg', 0), ('0002098114084658ab64a18cc6d92f57.jpg', 1)]


In [20]:
print(len(image_labels))

num_1s = 0
for img in image_labels:
    if img[1] == 1:
        num_1s += 1

print(num_1s)
# 53k 1s out of 83k images

83724
52900


In [22]:
# split data
#############################################
# How much train data do we want to use?
train_size = 10000
#############################################
test_size, val_size = 2000, 2000
train_data, val_data, test_data = split_data(
    image_labels, train_size, val_size, test_size, seed
)

# num_categories = len(image_labels[0][1])
num_categories = 1
print(num_categories)

1


In [23]:
print(len(test_data))
print(test_data[0])

print(len(val_data))
print(val_data[0])

print(len(train_data))
print(train_data[0])

# train_data data points vary with train_size, test and val remain constant with everything, as intended

2000
('5fa6c9837bad45c3bbda41c2d9067888.jpg', 1)
2000
('1b08dc061ccd4df787e837f6ce2a8f22.jpg', 1)
10000
('05ffba8e95b0410db8253e1c05340e2d.jpg', 1)


In [24]:
from utils import create_data_loaders

# create Torch DataLoaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
image_root_dir = data_path + "images/"
batch_size=32
train_loader, val_loader, test_loader = create_data_loaders(
    train_data, val_data, test_data, image_root_dir, batch_size, transform
)

In [25]:
from models import get_model

In [26]:
model_vgg = get_model("vgg16", tune_conv=True, num_categories=num_categories, dropout_rate=0.5)
print(model_vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [27]:
model_resnet = get_model("resnet50", tune_conv=True, num_categories=num_categories, dropout_rate=0.5)
print(model_resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
for name, param in model_vgg.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")

In [ ]:
for name, param in model_resnet.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")